# Assessing suitability of a location for thermal curtains and light shades to increase crop growth

## Introduction
Plants do not grow effectively and get can get damaged in high temperatures and too much sun. Conversely dull, cool days inhibit photosynthesis and plants do not grow effeciently, if temperatures drop too low grow may be damaged and leaves dropped impacting growth for many weeks.  Many greenhouse growers choose to whitewash the glass to reflect the heat and dissipate the light. However, adding white wash is time consuming, so the wash is only applied once per season, meaning if a cooler or cloudier spell of weather occurs the plants can't grow optimally. An increasingly popular alternative is fitting greenhouses with shading curtains which are easier to remove on cloudier days and can been drawn at night to retain heat. Through this project we investigate areas of xxxxxxxxxxxxxxxx to see white washing and thermal curtains are not efficient due to frequent temperature variations.

The meteostat platform has been used to access the weather data for the basis of this project [(1)](https://dev.meteostat.net/). Due to the amount of data it has been stored in a MySQL database and analysed using charts and predictions made using SciKitLearn.


# Plant shading techniques
Each of the different plant shading techniques have different characteristics and efficiencies. White washing requires good knowledge on when to white wash and remove it, but is very cheap, thermal curtains are easier to use, but are more expensive to buy.

## White washing
White washing is best applied to green houses when the days are consistently above 25 degrees and removed at the end of the season. The white washing provides limited thermal properties at night and hence only day time highs are of importance. The standard white washing protocol reduces daytime temperatures by 6.8% on sunny days.

In [10]:
whitewash = 6.8/100

### Thermal curtains
Thermal curtains can easily be pulled and retracted in line with local conditions. A shade rated for 50% shading gives the most similar effects in daytime to white washing and this reduced the daytime temperatures by 6.9% on sunny days and maintained over night temperatures by 2.1%.

In [11]:
curtains_day = 6.9/100
curtains_night = 2.1/100

### The test plants
"Head lettuce is the most important salad vegetable gown in the United States" [(2)](https://content.ces.ncsu.edu/lettuce). The optimum growing temperature for head lettuce is 15-18°C. At 20-26°C the plants start to flower and produce seed and need replacing and temperatures below 0°C will kill the plants.


In [12]:
lettuce_min = 0
lettuce_max = 20
lettuce_range1 = 15
lettuce_range2 = 20

Tomatoes are grown across the world and are an important staple in many cuisines. To increase their range and season many are grown in greenhouses. The optimal conditions for growth are 21-27°C with temperatures not dropping below 16°C or exceeding 29°C [(3)](https://drygair.com/blog/what-are-the-ideal-conditions-for-greenhouse-tomatoes/).

In [13]:
tomato_min = 16
tomato_max = 29
tomato_range1 = 21
tomato_range2 = 27

## Import needed tools

In [2]:
# Create the mySQL database
import mysql.connector as msql

# Import the data and unzip it
import requests
import gzip
from io import BytesIO
import json

# Create the dataframe and manipulate it
import pandas as pd
import datetime as dt

# Plot the data
import matplotlib.pyplot as plt
import seaborn as sns


# Create a database to store the data
Due to the large amounts of data needed for this analysis. A MySQL database needs to be created. MySQL offers the ability to store very large amounts of data in a structured and easily accessible way, where the data is persistent. 

To create the database we first set the connection parameters for the database and create the cursor to enable the connection for making SQL queries.


In [3]:
# Set the parameters for the connection
db = msql.connect(host='localhost', user='root',password='')

# Create the cursor
cursor = db.cursor()

# Execute SQL query
cursor.execute('CREATE DATABASE weather')

# Close the connection
cursor.close()
db.close()

: 

Within the database a table needs to be created for each of the data sources. The first table will hold a list of the weather stations [(4)](https://dev.meteostat.net/bulk/stations.html#endpoints)

In [ ]:
try:
    # Connect to MySQL
    con = msql.connect(host='localhost', database='weather', user='root', password='')

    if con.is_connected():
        cursor = con.cursor()
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print(f"You're connected to database: {record}")
        
        # Drop the table if it exists
        cursor.execute('DROP TABLE IF EXISTS stations')
        print('Creating table....')
        
        # Create the table
        sql = """CREATE TABLE stations 
            (id VARCHAR(10) PRIMARY KEY, 
            name JSON, 
            country CHAR(2), 
            region VARCHAR(10), 
            national_id VARCHAR(10), 
            wmo_id VARCHAR(10), 
            icao_id VARCHAR(10), 
            iata_id VARCHAR(25), 
            latitude DECIMAL(9,6), 
            longitude DECIMAL(9,6), 
            elevation INT, 
            timezone VARCHAR(25),
            history JSON, 
            hourly_start DATE, 
            hourly_end DATE, 
            daily_start DATE,
            daily_end DATE,
            monthly_start YEAR,
            monthly_end YEAR,
            normals_start YEAR,
            normals_end YEAR
)"""
        cursor.execute(sql)
        print("Table is created....")

except msql.Error as err:
    print(err.msg)

finally:
    if con.is_connected():
        cursor.close()
        con.close()

Then the weather stations table needs to be populated.

In [ ]:
# Step 1: Download and Decompress the Data
url = "https://bulk.meteostat.net/v2/stations/full.json.gz"
file_name = "full.json.gz"

# Download the file
response = requests.get(url, stream=True)
with open(file_name, 'wb') as f:
    f.write(response.content)

# Decompress the file
with gzip.open(file_name, 'rt', encoding='utf-8') as gz_file:
    stations_data = json.load(gz_file)

# Step 2: Connect to the MySQL Database
try:
    con = msql.connect(host='localhost', database='weather', user='root', password='')
    if con.is_connected():
        cursor = con.cursor()
        print("Connected to the database.")

        # Prepare SQL statement for data insertion
        sql_insert = """
            INSERT INTO stations (
                id, name, country, region, national_id, wmo_id, icao_id, iata_id,
                latitude, longitude, elevation, timezone, history,
                hourly_start, hourly_end, daily_start, daily_end,
                monthly_start, monthly_end, normals_start, normals_end
            ) VALUES (
                %(id)s, %(name)s, %(country)s, %(region)s, %(national_id)s, %(wmo_id)s, %(icao_id)s, %(iata_id)s,
                %(latitude)s, %(longitude)s, %(elevation)s, %(timezone)s, %(history)s,
                %(hourly_start)s, %(hourly_end)s, %(daily_start)s, %(daily_end)s,
                %(monthly_start)s, %(monthly_end)s, %(normals_start)s, %(normals_end)s
            )
        """

        # Step 3: Insert Data into MySQL Table
        for station in stations_data:
            data = {
                'id': station.get('id'),
                'name': json.dumps(station.get('name', {})),  # Convert dictionary to JSON string
                'country': station.get('country'),
                'region': station.get('region'),
                'national_id': station.get('national'),
                'wmo_id': station.get('wmo'),
                'icao_id': station.get('icao'),
                'iata_id': station.get('iata'),
                'latitude': station.get('latitude'),
                'longitude': station.get('longitude'),
                'elevation': station.get('elevation'),
                'timezone': station.get('timezone'),
                'history': json.dumps(station.get('history', [])),  # Convert list to JSON string
                'hourly_start': station.get('inventory', {}).get('hourly', {}).get('start'),
                'hourly_end': station.get('inventory', {}).get('hourly', {}).get('end'),
                'daily_start': station.get('inventory', {}).get('daily', {}).get('start'),
                'daily_end': station.get('inventory', {}).get('daily', {}).get('end'),
                'monthly_start': station.get('inventory', {}).get('monthly', {}).get('start'),
                'monthly_end': station.get('inventory', {}).get('monthly', {}).get('end'),
                'normals_start': station.get('inventory', {}).get('normals', {}).get('start'),
                'normals_end': station.get('inventory', {}).get('normals', {}).get('end'),
            }
            cursor.execute(sql_insert, data)

        # Commit the transaction
        con.commit()
        print("Data inserted successfully.")

except msql.Error as err:
    print(f"Error: {err}")

finally:
    if con.is_connected():
        cursor.close()
        con.close()
        print("MySQL connection is closed.")

## Select the country of interest


In [ ]:
con = msql.connect(host='localhost', database='weather', user='root', password='')

countries = pd.read_sql_query("SELECT DISTINCT country FROM stations",
    con)
print(countries)

## Create a table for the weather data for each region

In [ ]:
try:
    # Connect to MySQL
    con = msql.connect(host='localhost', database='weather', user='root', password='')

    if con.is_connected():
        cursor = con.cursor()
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print(f"You're connected to database: {record}")
        
        # Drop the table if it exists
        cursor.execute('DROP TABLE IF EXISTS stations')
        print('Creating table....')
        
        cursor.execute('DROP TABLE IF EXISTS ontario_data')
        print('Creating table....')
        cursor.execute("CREATE TABLE ontario_data(station_ID varchar(8), date TIMESTAMP, average_temp float, min_temp float, max_temp float, month int, year int)")
        print("Table is created....")

except msql.Error as err:
    print(err.msg)

finally:
    if con.is_connected():
        cursor.close()
        con.close()

## Read in the weather station data
The data about all the weather stations is available at: https://bulk.meteostat.net/v2/stations/lite.json.gz. It is compressed using `gzip` so they file needs to be decompressed before creating the dataframe

### Create variables to make it easier to choose the country and region of interest

host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

conn = pymysql.connect(
    host=host,
    port=int(port),
    user=user,
    passwd=password,
    db=database,
    charset='utf8mb4')

In [ ]:
country = df['country'].unique()
print(f"The countries within the file are:\n {country}")


## Prompt user to request the location of interest


In [9]:
chosencountry = input("Enter the country of interest: ")


In [10]:
countrydf = df[df['country'] == chosencountry]

In [ ]:
print(countrydf['region'].unique())

In [12]:
chosenregion = input("Enter the region of interest: ") 

## Create a new dataframe of the weather stations in the country and region of interest

In [ ]:
areaofinterest = df[df["region"] == chosenregion]
print(areaofinterest.head)

In [14]:
listofstations = areaofinterest['id'].unique()

## Pass in each weather station ID


### Create column names

In [ ]:
for station in listofstations:
    url = "https://bulk.meteostat.net/v2/daily/" + station + ".csv.gz"
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful

    # Decompress the gzip file
    data = gzip.GzipFile(fileobj=BytesIO(response.content))

    # Read in the CSV file
    df = pd.read_csv(data, usecols=[0,1,2,3],names =['Date', 'tavg', 'tmin', 'tmax'], parse_dates =['Date'])



In [ ]:
# Create columns for the year and month of the data
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
print(df.info())
print(df.head(10))

## Weather station data for weather station 1211
https://dev.meteostat.net/bulk/daily.html#endpoints

## Average the data by month
First check if there is any missing data.

In [ ]:
print(df.shape)
count_nan = df.isnull().sum()
print(count_nan)

If there is missing data use `dropna` to remove those rows.

### Plot the data for the average temperature


In [ ]:
# Step 3: Group by year and month, then calculate the average temperature for each month
df = df.dropna(subset=["tavg"])  # Remove rows with missing average temperature
monthly_avg = df.groupby(["year", "month"])["tavg"].mean().reset_index()

# Step 4: Plot the data
plt.figure(figsize=(12, 6))
years = monthly_avg["year"].unique()

for year in years:
    year_data = monthly_avg[monthly_avg["year"] == year]
    plt.plot(year_data["month"], year_data["tavg"], marker="o", label=str(year))


plt.title("Average Monthly Temperature")
plt.xlabel("Month")
plt.ylabel("Average Temperature (°C)")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
plt.legend(title="Year", loc="upper left")
plt.grid(True)
plt.show()


In [ ]:
# Step 3: Group by year and month, then calculate the average temperature for each month
df = df.dropna(subset=["tmax"])  # Remove rows with missing average temperature
monthly_avg = df.groupby(["year", "month"])["tmax"].mean().reset_index()

# Step 4: Plot the data
plt.figure(figsize=(12, 6))
years = monthly_avg["year"].unique()

for year in years:
    year_data = monthly_avg[monthly_avg["year"] == year]
    plt.plot(year_data["month"], year_data["tmax"], marker="o", label=str(year))


# Add horizontal lines at 23°C and 15°C
plt.axhline(y=23, color='red', linestyle='--', linewidth=1, label='23°C')
plt.axhline(y=15, color='blue', linestyle='--', linewidth=1, label='15°C')


plt.title("Maximum Monthly Temperature")
plt.xlabel("Month")
plt.ylabel("Maximum Monthly Temperature (°C)")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
plt.legend(title="Year", loc="upper left")
plt.grid(True)
plt.show()


In [ ]:
monthly_avg.plot(x='Date', kind='line')
plt.show()


## Regression and Scikit Learn
https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from datetime import timedelta
import matplotlib.pyplot as plt

# Step 1: Load Data
url = "https://bulk.meteostat.net/v2/daily/0CNUO.csv.gz"
df = pd.read_csv(url, compression='gzip', usecols=[0,1,2,3], names =["date", "tavg", "tmin", "tmax"])

# Step 2: Select Relevant Columns
df.dropna(inplace=True)

# Step 3: Prepare Features
df['date'] = pd.to_datetime(df['date'])
df['day_of_year'] = df['date'].dt.dayofyear
df['year'] = df['date'].dt.year

# Features: Day of the year and year; Target: Average temperature
X = df[['day_of_year', 'year']]
y = df['tavg']

# Step 4: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train Model
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Step 6: Evaluate Model
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error: {rmse:.2f}°C")

# Step 7: Predict for Next 5 Years
future_years = pd.date_range(start=df['date'].max() + timedelta(days=1), 
                             periods=5 * 365, freq='D')
future_df = pd.DataFrame({
    'date': future_years,
    'day_of_year': future_years.day_of_year,
    'year': future_years.year
})
future_predictions = model.predict(future_df[['day_of_year', 'year']])
future_df['predicted_tavg'] = future_predictions

# Step 8: Plot Known Data and Predictions
plt.figure(figsize=(14, 7))

# Plot historical data
plt.plot(df['date'], df['tavg'], label='Historical Data', alpha=0.6)

# Plot predictions
plt.plot(future_df['date'], future_df['predicted_tavg'], label='Predictions', alpha=0.8)

plt.title('Historical and Predicted Average Temperatures')
plt.xlabel('Date')
plt.ylabel('Average Temperature (°C)')
plt.legend()
plt.grid()
plt.show()

# Step 9: Save Predictions
future_df.to_csv('predicted_temperatures.csv', index=False)
print(f"Predictions saved to 'predicted_temperatures.csv'")

# Sci Kit Learn
https://www.askpython.com/python/examples/weather-data-clustering

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from itertools import cycle, islice
from pandas.plotting import parallel_coordinates

%matplotlib inline


## Load the dataset

In [ ]:
date = "Date"
tavg = "Average air temperature in Celsius"
tmin = "Minimum air temperature in Celsius"
tmax = "Maximum air temperature in Celsius"

station1211 = pd.read_csv("https://bulk.meteostat.net/v2/daily/0CNUO.csv.gz", usecols = [0,1,2,3], names =[date, tavg, tmin, tmax],parse_dates=[0])
#station1211["id"] = df.index + 1
print(station1211.info())

## Sample the data

In [ ]:
sampled_df = station1211[(station1211['Date'] % 2) == 0]
sampled_df.shape

#sampled_df.set_index('Date', inplace=True)

# del sampled_df['Average air temperature in Celsius']
# del sampled_df['Maximum air temperature in Celsius']
 


In [ ]:
B = sampled_df.shape[0]
sampled_df = sampled_df.dropna()
A = sampled_df.shape[0]
 
print("No of rows deleted: ", B-A)

In [ ]:
# Required Libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from datetime import datetime

# Step 1: Load Data
url = "https://bulk.meteostat.net/v2/daily/0CNUO.csv.gz"
df = pd.read_csv(url, compression='gzip', usecols=[0,1,2,3], names =[date, tavg, tmin, tmax])
df.info()

# Step 2: Select First Four Columns
#df = df[0,1,2,3]

# Step 3: Handle Missing Values
df.dropna(inplace=True)

# Step 4: Add Cyclical Features for Date
df['Date'] = pd.to_datetime(df['Date'])
df['day_of_year'] = df['Date'].dt.dayofyear
df['sin_day'] = np.sin(2 * np.pi * df['day_of_year'] / 365.0)
df['cos_day'] = np.cos(2 * np.pi * df['day_of_year'] / 365.0)

# Step 5: Prepare Data for Clustering
features = ['tavg', 'tmin', 'tmax', 'sin_day', 'cos_day']
X = df[features]

# Step 6: Apply KMeans
kmeans = KMeans(n_clusters=4, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Step 7: Visualize Clusters
plt.figure(figsize=(10, 6))
for cluster in range(kmeans.n_clusters):
    cluster_data = df[df['cluster'] == cluster]
    plt.scatter(cluster_data['day_of_year'], cluster_data['tavg'], label=f'Cluster {cluster}')
    
plt.title('KMeans Clustering of Weather Data')
plt.xlabel('Day of Year')
plt.ylabel('Average Temperature (°C)')
plt.legend()
plt.show()


## Thermoreactive glass

The data in the below has been changed to help protect the intellectual property. Many plant crops scorch and reduce their efficiency at temperatures over 23 degrees and the traditional approach has been to whitewash growhouses. Unfortunately, this white washing is not easily removed, so whitewashes are applied at the beginning of the summer and washed off at the end, this means if there is a run of poor weather in summer the plants can't be productive and if there is good weather outside of summer the plants scorch. 

In [ ]:
# Highlight data for days with tmax > 23 degrees and proximity to tmax < 17 degrees
# Step 5: Identify days with tmax > 23 degrees
df["hot_days"] = df["tmax"] > 23

# Step 6: Identify days within 5 days of a day with tmax < 17 degrees
df["cold_days"] = df["tmax"] < 17
df["cold_within_5"] = df["cold_days"].rolling(window=5, min_periods=1).max().shift(-4).fillna(0).astype(bool)

df["hot_near_cold"] = df["hot_days"] & df["cold_within_5"]

# Step 7: Group by month and count hot days and hot days near cold days
highlight_summary = df.groupby("month").agg(
    total_hot_days=("hot_days", "sum"),
    hot_near_cold_days=("hot_near_cold", "sum")
).reset_index()

# Highlight data for days with tmax > 23 degrees and proximity to tmax < 17 degrees
# Step 5: Identify days with tmax > 23 degrees
df["hot_days"] = df["tmax"] > 23

# Step 6: Identify days within 5 days of a day with tmax < 17 degrees
df["cold_days"] = df["tmax"] < 17
df["cold_within_5"] = df["cold_days"].rolling(window=5, min_periods=1).max().shift(-4).fillna(0).astype(bool)

df["hot_near_cold"] = df["hot_days"] & df["cold_within_5"]

# Step 7: Group by year and month to count hot days and hot days near cold days
highlight_summary = df.groupby(["year", "month"]).agg(
    total_hot_days=("hot_days", "sum"),
    hot_near_cold_days=("hot_near_cold", "sum")
).reset_index()

# Step 8: Plot hot days and hot days near cold days
plt.figure(figsize=(12, 6))
for year in highlight_summary["year"].unique():
    year_data = highlight_summary[highlight_summary["year"] == year]
    plt.plot(year_data["month"], year_data["total_hot_days"], marker="o", label=f"Hot Days {year}")
    plt.plot(year_data["month"], year_data["hot_near_cold_days"], marker="x", label=f"Hot Near Cold {year}")

plt.title("Monthly Hot Days and Hot Near Cold Days")
plt.xlabel("Month")
plt.ylabel("Number of Days")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
plt.legend(title="Year", loc="upper left")
plt.grid(True)
plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression

# Step 9: Predict trends for the next five years
# Prepare data for regression analysis
highlight_summary["time_index"] = highlight_summary["year"] + (highlight_summary["month"] - 1) / 12.0

# Predict total_hot_days
X = highlight_summary[["time_index"]]
y_hot_days = highlight_summary["total_hot_days"]
hot_days_model = LinearRegression().fit(X, y_hot_days)

# Predict hot_near_cold_days
y_hot_near_cold_days = highlight_summary["hot_near_cold_days"]
hot_near_cold_model = LinearRegression().fit(X, y_hot_near_cold_days)

# Generate future predictions
future_years = np.arange(highlight_summary["year"].max() + 1, highlight_summary["year"].max() + 6)
future_months = np.tile(range(1, 13), len(future_years))
future_years_repeated = np.repeat(future_years, 12)
future_time_index = future_years_repeated + (future_months - 1) / 12.0

future_X = pd.DataFrame({"time_index": future_time_index})
future_hot_days = hot_days_model.predict(future_X)
future_hot_near_cold_days = hot_near_cold_model.predict(future_X)

# Combine predictions into a DataFrame
future_predictions = pd.DataFrame({
    "year": future_years_repeated,
    "month": future_months,
    "predicted_hot_days": future_hot_days,
    "predicted_hot_near_cold_days": future_hot_near_cold_days
})

# Display predictions
print(future_predictions)

# Plot predictions
plt.figure(figsize=(12, 6))
for year in future_years:
    year_data = future_predictions[future_predictions["year"] == year]
    plt.plot(year_data["month"], year_data["predicted_hot_days"], marker="o", label=f"Predicted Hot Days {year}")
    plt.plot(year_data["month"], year_data["predicted_hot_near_cold_days"], marker="x", label=f"Predicted Hot Near Cold {year}")

plt.title("Predicted Monthly Hot Days and Hot Near Cold Days")
plt.xlabel("Month")
plt.ylabel("Number of Days")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
plt.legend(title="Year", loc="upper left")
plt.grid(True)
plt.show()


[(1)]https://dev.meteostat.net/
[(2)] https://content.ces.ncsu.edu/lettuce
[(3)] https://drygair.com/blog/what-are-the-ideal-conditions-for-greenhouse-tomatoes/
https://www.askpython.com/python/examples/weather-data-clustering

https://esciencecenter-digital-skills.github.io/lesson-machine-learning-intro/02-data-exploration.html